In [52]:
import importlib
import os
from models import *
from dataprocessor import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)

In [77]:
def check_for_nan(tensors):
    """检查给定张量列表中是否有NaN值"""
    for tensor in tensors:
        if tf.reduce_any(tf.math.is_nan(tensor)):
            print("NAN in dataset")
            return True
        if tf.reduce_any(tf.math.is_inf(tensor)):
            print("inf in dataset")  
            return True
    return False

In [110]:
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx", "SP00.CME.xlsx")]
data_list = []


data_list = process_data_list(files, macd_timescales, rtn_timescales, test = False)
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
context_set, _ = generate_tensors(data_list, timesteps, encoder_type = "one-hot", contain_next_day_rtn=True)

生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:34<00:00,  3.10s/it]


one-hot 编码中...


生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:33<00:00,  3.02s/it]


one-hot 编码中...


In [111]:
batch_size = len(files)
data_set = data_binder(context_set, target_set, labels, batch_size=batch_size)
data_set.save(f"saved_data")

dataset = tf.data.Dataset.load("saved_data")

处理日期中..: 100%|██████████| 86898/86898 [36:38<00:00, 39.52it/s]


In [112]:
reload_custom_libs()
from models import *
from dataprocessor import *
from loss_functions import *
# from heartrate import trace

hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(files) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

In [113]:
if tf.reduce_any(tf.math.is_inf(x)):
    print("NAN in dataset")

NAN in dataset


In [115]:
iter_count, num = 0, 0
for batch_data in tqdm(dataset, desc=f"训练中...epoch"):
    x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = batch_data
    if check_for_nan([x_c, x_c_rtn, s_c, s, rtn_std]):
        num += 1
        print(f"detected in batch_data, iteration {iter_count}")
    iter_count += 1

训练中...epoch: 100%|██████████| 2825/2825 [00:04<00:00, 694.38it/s]


In [ ]:
import tensorflow as tf

class ScaledDotProductAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def call(self, q, k, v):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # 计算 softmax 权重
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

        # 计算注意力输出
        output = tf.matmul(attention_weights, v)

        return output, attention_weights

def print_tensor(tensor):
    print(tensor.numpy())

q = tf.random.uniform((1, 3, 64))  # query: batch_size=1, seq_len=3, depth=64
k = tf.random.uniform((1, 5, 64))  # key: batch_size=1, seq_len=5, depth=64
v = tf.random.uniform((1, 5, 64))  # value: batch_size=1, seq_len=5, depth=64

attention_layer = ScaledDotProductAttention()
output, attention_weights = attention_layer(q, k, v)

print_tensor(output)
print_tensor(attention_weights)


In [116]:
num_epochs = 100
print_interval = 10

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

def train():
    
    dataset = tf.data.Dataset.load("saved_data")
    for epoch in range(num_epochs):
        dataset = dataset.shuffle(buffer_size=10000)
        iter_count = 0
        for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
            x_c, x_c_rtn, s_c, x, s, rtn_std, _, _ = batch_data
            if check_for_nan([x_c, x_c_rtn, s_c, x, s, rtn_std]):
                print(f"NaN or inf detected in batch_data, iteration {iter_count}")
                return batch_data
            with tf.GradientTape() as tape:
                result = model(x_c, x_c_rtn, s_c, x, s)
                if check_for_nan(result):
                    print(f"NaN detected in result, iteration {iter_count}")
                    return batch_data, result
                loss, mle, sharpe = joint_loss_function(result, rtn_std, target_std, warm_up, alpha=1)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            iter_count += 1
            if iter_count % 1 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {loss.numpy()}, mle: {mle}, sharpe: {sharpe}")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.numpy()}")
    
result = train()

训练中...epoch0:   0%|          | 0/2825 [00:00<?, ?it/s]

训练中...epoch0:   0%|          | 1/2825 [00:05<4:24:37,  5.62s/it]

Epoch 1/100, Iteration 1, Loss: 12147.961673654874, mle: 12150.020922171767, sharpe: -2.059248516893073


训练中...epoch0:   0%|          | 2/2825 [00:08<2:56:34,  3.75s/it]

Epoch 1/100, Iteration 2, Loss: 2020.2567054060887, mle: 2021.0592228756884, sharpe: -0.8025174695996306


...

Epoch 2/100, Iteration 40, Loss: 473.7466834417068, mle: 472.2176577534672, sharpe: 1.529025688239597


训练中...epoch1:   1%|▏         | 41/2825 [01:52<1:56:17,  2.51s/it]

Epoch 2/100, Iteration 41, Loss: 2118.8926818695663, mle: 2124.3519722089613, sharpe: -5.459290339395079


训练中...epoch1:   1%|▏         | 41/2825 [01:54<2:09:09,  2.78s/it]


KeyboardInterrupt: 

In [ ]:
result

(<tf.Tensor: shape=(12, 126, 2), dtype=float64, numpy=
 array([[[6.11744404, 1.29992247],
         [6.05334282, 1.19234073],
         [6.04828453, 1.12541842],
         ...,
         [6.06719971, 0.9283601 ],
         [6.0078578 , 0.92848593],
         [6.02805185, 0.93377763]],
 
        [[1.25681174, 1.21873188],
         [1.18810105, 1.17738163],
         [1.14196587, 1.16670406],
         ...,
         [1.28490782, 1.0845989 ],
         [1.2713486 , 1.11892009],
         [1.2127372 , 1.10908401]],
 
        [[4.17370129, 1.50630212],
         [4.08983517, 1.42670906],
         [4.06532669, 1.39577425],
         ...,
         [3.21722984, 1.31997609],
         [3.23926163, 1.31692803],
         [3.22620964, 1.32057881]],
 
        ...,
 
        [[3.89128351, 2.25793195],
         [3.79784536, 2.07032108],
         [3.76586294, 1.93990648],
         ...,
         [3.76935387, 1.57853842],
         [3.73729277, 1.56849384],
         [3.7743268 , 1.56140959]],
 
        [[6.20525122, 

In [ ]:
batch_data, result

((<tf.Tensor: shape=(12, 126, 8), dtype=float32, numpy=
  array([[[ 4.22154129e-01,  5.78526735e-01,  6.94468319e-01, ...,
            7.02226639e-01,  1.66043854e+00,  1.39849818e+00],
          [ 2.72120357e-01,  4.73401397e-01,  6.11810982e-01, ...,
            8.59434485e-01,  1.87758172e+00,  1.46209788e+00],
          [ 1.50553852e-01,  3.85598093e-01,  5.29221356e-01, ...,
            6.16452217e-01,  2.23761201e+00,  1.47328079e+00],
          ...,
          [-1.35168362e+00, -9.07790482e-01, -3.40614229e-01, ...,
           -1.41744688e-01, -1.11088246e-01,  5.15854120e-01],
          [-1.36484730e+00, -9.87687826e-01, -4.50251132e-01, ...,
           -1.22196287e-01, -1.08352743e-01,  5.82782269e-01],
          [-1.32851899e+00, -1.02780652e+00, -5.31191885e-01, ...,
           -1.40540302e-01, -6.97354749e-02,  6.40299082e-01]],
  
         [[-7.60377884e-01, -1.01712239e+00, -1.06724250e+00, ...,
           -2.09264055e-01, -8.05742204e-01,  2.37559617e-01],
          [-7.1

In [ ]:
batch_data[-4][-2]

<tf.Tensor: shape=(126, 13), dtype=float32, numpy=
array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)>